This notebook was used to determine which model was best for this problem out of models with different orders and amounts of dilating cnn layers and basic cnn layers

In [ ]:
import time
import sys
import os
import glob
import math
import threading
import concurrent.futures as cf

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import Input, Model, layers, metrics, losses, callbacks, optimizers, models, utils
from keras import backend as K
import gc
import keras_tuner as kt
from pyfaidx import Fasta

K.clear_session()
gc.collect()

datasets_path = "../../Datasets/"
models_path = "../../Models/"

In [3]:
class CustomNonZeroF1Score(tf.keras.metrics.Metric):
    def __init__(self, num_classes, average='weighted', name='non_zero_f1', **kwargs):
        """
        Custom F1 score metric that only considers non-zero classes.
        
        Args:
            num_classes (int): Total number of classes. Class 0 is assumed to be the "background" class.
            average (str): 'weighted' (default) to weight by support or 'macro' for a simple average.
            name (str): Name of the metric.
            **kwargs: Additional keyword arguments.
        """
        super(CustomNonZeroF1Score, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        if average not in ['weighted', 'macro']:
            raise ValueError("average must be 'weighted' or 'macro'")
        self.average = average
        
        # Accumulate counts per class
        self.true_positives = self.add_weight(
            name='tp', shape=(num_classes,), initializer='zeros', dtype=tf.float32
        )
        self.false_positives = self.add_weight(
            name='fp', shape=(num_classes,), initializer='zeros', dtype=tf.float32
        )
        self.false_negatives = self.add_weight(
            name='fn', shape=(num_classes,), initializer='zeros', dtype=tf.float32
        )
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        """
        Updates the confusion matrix statistics.
        
        Args:
            y_true: Tensor of shape (batch_size, seq_length) with integer class labels.
            y_pred: Tensor of shape (batch_size, seq_length, num_classes) with probability distributions.
            sample_weight: Optional sample weights.
        """
        # Convert predictions to class labels using argmax along the last axis.
        y_pred = tf.argmax(y_pred, axis=-1)
        
        # Flatten the batch and sequence dimensions.
        y_true = tf.reshape(y_true, [-1])
        y_pred = tf.reshape(y_pred, [-1])
        
        # Compute confusion matrix over all predictions.
        cm = tf.math.confusion_matrix(
            y_true, y_pred, num_classes=self.num_classes, dtype=tf.float32
        )
        tp = tf.linalg.diag_part(cm)
        fp = tf.reduce_sum(cm, axis=0) - tp
        fn = tf.reduce_sum(cm, axis=1) - tp
        
        # Update state variables.
        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)
    
    def result(self):
        """
        Computes the F1 score for non-zero classes.
        
        Returns:
            F1 score computed over the non-zero classes.
        """
        precision = tf.math.divide_no_nan(
            self.true_positives, self.true_positives + self.false_positives
        )
        recall = tf.math.divide_no_nan(
            self.true_positives, self.true_positives + self.false_negatives
        )
        f1 = tf.math.divide_no_nan(2 * precision * recall, precision + recall)
        
        # Exclude class 0 (the background) from the evaluation.
        f1_non_zero = f1[1:]
        support_non_zero = (self.true_positives + self.false_negatives)[1:]
        
        if self.average == 'weighted':
            # Weight F1 by the support of each class.
            weighted_f1 = tf.reduce_sum(f1_non_zero * support_non_zero) / (tf.reduce_sum(support_non_zero) + K.epsilon())
            return weighted_f1
        else:  # macro
            return tf.reduce_mean(f1_non_zero)
    
    def reset_states(self):
        """
        Resets the metric state variables.
        """
        for v in self.variables:
            v.assign(tf.zeros_like(v))

In [4]:
def drop_exact_records(dataset: tf.data.Dataset, total_records, num_to_drop, seed=None):
    if seed:
        np.random.seed(seed)
    drop_indices = set(np.random.choice(total_records, num_to_drop, replace=False))
    dataset = dataset.enumerate()
    dataset = dataset.filter(lambda i, x: ~tf.reduce_any(tf.equal(i, list(drop_indices))))
    dataset = dataset.map(lambda i, x: x)
    return dataset


def parse_chunk_example(serialized_example):
    """
    Parses a single serialized tf.train.Example back into tensors.
    Used in testing datasets and in piping tfrecords to DL Algorithms
    """
    feature_spec = {
        'X':          tf.io.VarLenFeature(tf.float32),
        'y':          tf.io.VarLenFeature(tf.int64),
        'record_id':  tf.io.FixedLenFeature([], tf.string),
        'cstart':     tf.io.FixedLenFeature([1], tf.int64),
        'cend':       tf.io.FixedLenFeature([1], tf.int64),
        'strand':     tf.io.FixedLenFeature([], tf.string),
        'chunk_size': tf.io.FixedLenFeature([1], tf.int64),
    }
    
    parsed = tf.io.parse_single_example(serialized_example, feature_spec)
    
    # chunk_size is shape [1]
    chunk_size = parsed['chunk_size'][0]
    
    # Convert sparse to dense
    X_flat = tf.sparse.to_dense(parsed['X'])
    y_flat = tf.sparse.to_dense(parsed['y'])

    # Reshape X to [chunk_size, 5]
    X_reshaped = tf.reshape(X_flat, [chunk_size, 5])
    # Reshape y to [chunk_size], probably redundant
    y_reshaped = tf.reshape(y_flat, [chunk_size])
    
    record_id = parsed['record_id']
    cstart    = parsed['cstart'][0]
    cend      = parsed['cend'][0]
    strand    = parsed['strand']
    
    return X_reshaped, y_reshaped, record_id, cstart, cend, strand


def prepare_for_model(X, y, record_id, cstart, cend, strand):
    '''
    Extracts and reshapes parsed data for feeding to DL Models
    '''
    # Expand last dimension of y from (batch_size, 5000) to (batch_size, 5000, 1)
    # y = tf.expand_dims(y, axis=-1) turns out this line is not needed
    # Return only (X, y). Discard the extra columns for training knowing that 
    # they still exist in the TestValTrain originals if we need them
    return X, y


def prep_dataset_from_tfrecord(
    tfrecord_path,
    batch_size=32,
    compression_type='GZIP',
    shuffled = False,
    shuffle_buffer=100000,
    total_records=None,
    num_to_drop=None,
    seed=None
):
    '''
    Imports tfrecord and shuffles it then parses it for use in fitting a model
    '''
    # Loads in records in a round robin fashion for slightly increased mixing
    dataset = tf.data.TFRecordDataset(tfrecord_path, compression_type=compression_type, num_parallel_reads = tf.data.AUTOTUNE)
    
    if num_to_drop:
        dataset = drop_exact_records(dataset, total_records=total_records, num_to_drop=num_to_drop, seed=seed)
    
    if shuffled == True:
        # Shuffle at the record level
        dataset = dataset.shuffle(shuffle_buffer, reshuffle_each_iteration=True)
        
    
    dataset = dataset.map(parse_chunk_example, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(prepare_for_model, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(lambda x, y: (x, tf.cast(y, tf.int32))) # found out tensorflow wants int32 in y for sparse categorical crossentropy

    # Rebatch parsed and prefetch for efficient reading
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [5]:
def create_cnn_model(
    input_dim=5,
    sequence_length=5000,
    num_classes=5
):
    inputs = Input(shape=(sequence_length, input_dim))
    
    # 1) Create a range of positions [0..sequence_length-1]
    positions = tf.range(start=0, limit=sequence_length, delta=1)
    
    # 2) Embed them => shape = (sequence_length, num_classes)
    pos_encoding = layers.Embedding(input_dim=sequence_length, 
                                    output_dim=num_classes)(positions)
    
    # 3) Expand dims to have shape = (1, sequence_length, num_classes)
    pos_encoding = tf.expand_dims(pos_encoding, axis=0)
    
    # 4) Tile across the batch dimension.
    #    We do this via a Lambda that takes a list of [pos_encoding, inputs]
    #    so that we can read the symbolic batch size from `inputs`.
    def tile_to_batch(z):
        # z = [pos_encoding, inputs]
        pe, x = z
        # tile so that the 0th dimension is the batch size of 'x'
        return tf.tile(pe, [tf.shape(x)[0], 1, 1])
    
    pos_encoding = layers.Lambda(tile_to_batch)([pos_encoding, inputs])
    # shape: (batch_size, sequence_length, num_classes)

    # Concatenate original inputs with positional encoding along the last dimension
    concat_input = layers.Concatenate(axis=-1)([inputs, pos_encoding])

    # Convolutional blocks with dropout (pooling is bad because exact sequence matters)
    # Two of each filter count to keep size in line with the combined models
    cnn = layers.Conv1D(filters=128, kernel_size=5, activation='relu', padding='same')(concat_input)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(0.3)(cnn)
    
    cnn = layers.Conv1D(filters=128, kernel_size=5, activation='relu', padding='same')(cnn)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(0.3)(cnn)

    cnn = layers.Conv1D(filters=160, kernel_size=5, activation='relu', padding='same')(cnn)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(0.3)(cnn)
    
    cnn = layers.Conv1D(filters=160, kernel_size=5, activation='relu', padding='same')(cnn)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(0.3)(cnn)

    cnn = layers.Conv1D(filters=192, kernel_size=5, activation='relu', padding='same')(cnn)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(0.3)(cnn)
    
    cnn = layers.Conv1D(filters=192, kernel_size=5, activation='relu', padding='same')(cnn)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(0.3)(cnn)

    # Instead of flattening, use Conv1D with kernel_size=1 as dense layers:
    cnn = layers.Conv1D(128, kernel_size=1, activation='relu')(cnn)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(0.3)(cnn)
    
    cnn = layers.Conv1D(128, kernel_size=1, activation='relu')(cnn)
    cnn = layers.BatchNormalization()(cnn)

    # Final classification layer applied at every time step:
    outputs = layers.Conv1D(num_classes, kernel_size=1, activation='softmax')(cnn)

    model = Model(inputs=inputs, outputs=outputs)
    return model

cnn_model = create_cnn_model(5, 5000, 5)
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
cnn_model.summary()

I0000 00:00:1738657947.507522     734 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1738657947.713995     734 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1738657947.714107     734 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1738657947.718983     734 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1738657947.719084     734 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 5000, 5)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 5000, 5)   │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 5000, 10)  │          0 │ input_layer[0][0… │
│ (Concatenate)       │                   │            │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 5000, 128) │      6,528 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 5000, 128) │        512 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 5000, 128) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 5000, 128) │     82,048 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 128) │        512 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 5000, 128) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 5000, 160) │    102,560 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 160) │        640 │ conv1d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 5000, 160) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 5000, 160) │    128,160 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 160) │        640 │ conv1d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 5000, 160) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 5000, 192) │    153,792 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 192) │        768 │ conv1d_4[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 5000, 192) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 5000, 192) │    184,512 │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 192) │        768 │ conv1d_5[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 5000, 192) │          0 │ batch_normalizat

 Total params: 704,325 (2.69 MB)

 Trainable params: 701,893 (2.68 MB)

 Non-trainable params: 2,432 (9.50 KB)

In [6]:
def create_dcnn_model(
    input_dim=5,
    sequence_length=5000,
    num_classes=5
):
    inputs = Input(shape=(sequence_length, input_dim))
    
    # Condensed positional encoding block.  See cnn for description
    positions = tf.range(start=0, limit=sequence_length, delta=1)
    pos_encoding = layers.Embedding(input_dim=sequence_length, output_dim=num_classes)(positions)
    pos_encoding = tf.expand_dims(pos_encoding, axis=0)
    def tile_to_batch(z):
        pe, x = z
        return tf.tile(pe, [tf.shape(x)[0], 1, 1])
    pos_encoding = layers.Lambda(tile_to_batch)([pos_encoding, inputs])

    concat_input = layers.Concatenate(axis=-1)([inputs, pos_encoding])


    # We use six layers with increasing dilation rates to capture a wider receptive field.
    # Dilating convolutional blocks with dropout (pooling is bad because exact sequence matters)
    # Two of each filter count to keep size in line with the combined models
    dcnn = layers.Conv1D(filters=128, kernel_size=5, dilation_rate=1, activation='relu', padding='same')(concat_input)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(0.3)(dcnn)
    
    dcnn = layers.Conv1D(filters=128, kernel_size=5, dilation_rate=1, activation='relu', padding='same')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(0.3)(dcnn)
    
    dcnn = layers.Conv1D(filters=160, kernel_size=5, dilation_rate=2, activation='relu', padding='same')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(0.3)(dcnn)
    
    dcnn = layers.Conv1D(filters=160, kernel_size=5, dilation_rate=2, activation='relu', padding='same')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(0.3)(dcnn)
    
    dcnn = layers.Conv1D(filters=192, kernel_size=5, dilation_rate=4, activation='relu', padding='same')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(0.3)(dcnn)
    
    dcnn = layers.Conv1D(filters=192, kernel_size=5, dilation_rate=4, activation='relu', padding='same')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(0.3)(dcnn)

    # Instead of flattening, use Conv1D with kernel_size=1 as dense layers:
    dcnn = layers.Conv1D(128, kernel_size=1, activation='relu')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(0.3)(dcnn)
    
    dcnn = layers.Conv1D(128, kernel_size=1, activation='relu')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)

    # Final classification layer applied at every time step:
    outputs = layers.Conv1D(num_classes, kernel_size=1, activation='softmax')(dcnn)

    model = Model(inputs=inputs, outputs=outputs)
    return model

dcnn_model = create_dcnn_model(5, 5000, 5)
dcnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
dcnn_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 5000, 5)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 5000, 5)   │          0 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 5000, 10)  │          0 │ input_layer_1[0]… │
│ (Concatenate)       │                   │            │ lambda_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_9 (Conv1D)   │ (None, 5000, 128) │      6,528 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 128) │        512 │ conv1d_9[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 5000, 128) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_10 (Conv1D)  │ (None, 5000, 128) │     82,048 │ dropout_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 128) │        512 │ conv1d_10[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 5000, 128) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_11 (Conv1D)  │ (None, 5000, 160) │    102,560 │ dropout_8[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 160) │        640 │ conv1d_11[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 5000, 160) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_12 (Conv1D)  │ (None, 5000, 160) │    128,160 │ dropout_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 160) │        640 │ conv1d_12[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 5000, 160) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_13 (Conv1D)  │ (None, 5000, 192) │    153,792 │ dropout_10[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 192) │        768 │ conv1d_13[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 5000, 192) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_14 (Conv1D)  │ (None, 5000, 192) │    184,512 │ dropout_11[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 192) │        768 │ conv1d_14[0][0]   │
│ (BatchNormalizatio… │                   │            │                 

 Total params: 704,325 (2.69 MB)

 Trainable params: 701,893 (2.68 MB)

 Non-trainable params: 2,432 (9.50 KB)

In [7]:
def create_cnn_to_dcnn_model(
    input_dim=5,
    sequence_length=5000,
    num_classes=5
):
    inputs = Input(shape=(sequence_length, input_dim))
    
    # Condensed positional encoding block.  See cnn for description
    positions = tf.range(start=0, limit=sequence_length, delta=1)
    pos_encoding = layers.Embedding(input_dim=sequence_length, output_dim=num_classes)(positions)
    pos_encoding = tf.expand_dims(pos_encoding, axis=0)
    def tile_to_batch(z):
        pe, x = z
        return tf.tile(pe, [tf.shape(x)[0], 1, 1])
    pos_encoding = layers.Lambda(tile_to_batch)([pos_encoding, inputs])

    concat_input = layers.Concatenate(axis=-1)([inputs, pos_encoding])

    # Convolutional blocks with dropout (pooling is bad because exact sequence matters):
    cnn = layers.Conv1D(filters=128, kernel_size=5, activation='relu', padding='same')(concat_input)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(0.3)(cnn)

    cnn = layers.Conv1D(filters=160, kernel_size=5, activation='relu', padding='same')(cnn)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(0.3)(cnn)

    cnn = layers.Conv1D(filters=192, kernel_size=5, activation='relu', padding='same')(cnn)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(0.3)(cnn)

    dcnn = layers.Conv1D(filters=128, kernel_size=5, dilation_rate=1, activation='relu', padding='same')(cnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(0.3)(dcnn)
    
    dcnn = layers.Conv1D(filters=160, kernel_size=5, dilation_rate=2, activation='relu', padding='same')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(0.3)(dcnn)
    
    dcnn = layers.Conv1D(filters=192, kernel_size=5, dilation_rate=4, activation='relu', padding='same')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(0.3)(dcnn)

    # Instead of flattening, use Conv1D with kernel_size=1 as dense layers:
    dcnn = layers.Conv1D(128, kernel_size=1, activation='relu')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(0.3)(dcnn)
    
    dcnn = layers.Conv1D(128, kernel_size=1, activation='relu')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)

    # Final classification layer applied at every time step:
    outputs = layers.Conv1D(num_classes, kernel_size=1, activation='softmax')(dcnn)

    model = Model(inputs=inputs, outputs=outputs)
    return model

cnn_to_dcnn_model = create_cnn_to_dcnn_model(5, 5000, 5)
cnn_to_dcnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
cnn_to_dcnn_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 5000, 5)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_2 (Lambda)   │ (None, 5000, 5)   │          0 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 5000, 10)  │          0 │ input_layer_2[0]… │
│ (Concatenate)       │                   │            │ lambda_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_18 (Conv1D)  │ (None, 5000, 128) │      6,528 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 128) │        512 │ conv1d_18[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_14          │ (None, 5000, 128) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_19 (Conv1D)  │ (None, 5000, 160) │    102,560 │ dropout_14[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 160) │        640 │ conv1d_19[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_15          │ (None, 5000, 160) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_20 (Conv1D)  │ (None, 5000, 192) │    153,792 │ dropout_15[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 192) │        768 │ conv1d_20[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_16          │ (None, 5000, 192) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_21 (Conv1D)  │ (None, 5000, 128) │    123,008 │ dropout_16[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 128) │        512 │ conv1d_21[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_17          │ (None, 5000, 128) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_22 (Conv1D)  │ (None, 5000, 160) │    102,560 │ dropout_17[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 160) │        640 │ conv1d_22[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_18          │ (None, 5000, 160) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_23 (Conv1D)  │ (None, 5000, 192) │    153,792 │ dropout_18[0][0]

 Total params: 688,965 (2.63 MB)

 Trainable params: 686,533 (2.62 MB)

 Non-trainable params: 2,432 (9.50 KB)

In [8]:
def create_dcnn_to_cnn_model(
    input_dim=5,
    sequence_length=5000,
    num_classes=5
):
    inputs = Input(shape=(sequence_length, input_dim))
    
    # Condensed positional encoding block.  See cnn for description
    positions = tf.range(start=0, limit=sequence_length, delta=1)
    pos_encoding = layers.Embedding(input_dim=sequence_length, output_dim=num_classes)(positions)
    pos_encoding = tf.expand_dims(pos_encoding, axis=0)
    def tile_to_batch(z):
        pe, x = z
        return tf.tile(pe, [tf.shape(x)[0], 1, 1])
    pos_encoding = layers.Lambda(tile_to_batch)([pos_encoding, inputs])

    concat_input = layers.Concatenate(axis=-1)([inputs, pos_encoding])

    # Convolutional blocks with dropout (pooling is bad because exact sequence matters):
    dcnn = layers.Conv1D(filters=128, kernel_size=5, dilation_rate=1, activation='relu', padding='same')(concat_input)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(0.3)(dcnn)
    
    dcnn = layers.Conv1D(filters=160, kernel_size=5, dilation_rate=2, activation='relu', padding='same')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(0.3)(dcnn)
    
    dcnn = layers.Conv1D(filters=192, kernel_size=5, dilation_rate=4, activation='relu', padding='same')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(0.3)(dcnn)
    
    cnn = layers.Conv1D(filters=128, kernel_size=5, activation='relu', padding='same')(dcnn)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(0.3)(cnn)

    cnn = layers.Conv1D(filters=160, kernel_size=5, activation='relu', padding='same')(cnn)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(0.3)(cnn)

    cnn = layers.Conv1D(filters=192, kernel_size=5, activation='relu', padding='same')(cnn)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(0.3)(cnn)

    # Instead of flattening, use Conv1D with kernel_size=1 as dense layers:
    cnn = layers.Conv1D(128, kernel_size=1, activation='relu')(cnn)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(0.3)(cnn)
    
    cnn = layers.Conv1D(128, kernel_size=1, activation='relu')(cnn)
    cnn = layers.BatchNormalization()(cnn)

    # Final classification layer applied at every time step:
    outputs = layers.Conv1D(num_classes, kernel_size=1, activation='softmax')(cnn)

    model = Model(inputs=inputs, outputs=outputs)
    return model

dcnn_to_cnn_model = create_dcnn_to_cnn_model(5, 5000, 5)
dcnn_to_cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
dcnn_to_cnn_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 5000, 5)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_3 (Lambda)   │ (None, 5000, 5)   │          0 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 5000, 10)  │          0 │ input_layer_3[0]… │
│ (Concatenate)       │                   │            │ lambda_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_27 (Conv1D)  │ (None, 5000, 128) │      6,528 │ concatenate_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 128) │        512 │ conv1d_27[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_21          │ (None, 5000, 128) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_28 (Conv1D)  │ (None, 5000, 160) │    102,560 │ dropout_21[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 160) │        640 │ conv1d_28[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_22          │ (None, 5000, 160) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_29 (Conv1D)  │ (None, 5000, 192) │    153,792 │ dropout_22[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 192) │        768 │ conv1d_29[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_23          │ (None, 5000, 192) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_30 (Conv1D)  │ (None, 5000, 128) │    123,008 │ dropout_23[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 128) │        512 │ conv1d_30[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_24          │ (None, 5000, 128) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_31 (Conv1D)  │ (None, 5000, 160) │    102,560 │ dropout_24[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 160) │        640 │ conv1d_31[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_25          │ (None, 5000, 160) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_32 (Conv1D)  │ (None, 5000, 192) │    153,792 │ dropout_25[0][0]

 Total params: 688,965 (2.63 MB)

 Trainable params: 686,533 (2.62 MB)

 Non-trainable params: 2,432 (9.50 KB)

In [9]:
def create_parallel_hybrid_model(
    input_dim=5,
    sequence_length=5000,
    num_classes=5
):
    inputs = Input(shape=(sequence_length, input_dim))
    
    # Condensed positional encoding block.  See cnn for description
    positions = tf.range(start=0, limit=sequence_length, delta=1)
    pos_encoding = layers.Embedding(input_dim=sequence_length, output_dim=num_classes)(positions)
    pos_encoding = tf.expand_dims(pos_encoding, axis=0)
    def tile_to_batch(z):
        pe, x = z
        return tf.tile(pe, [tf.shape(x)[0], 1, 1])
    pos_encoding = layers.Lambda(tile_to_batch)([pos_encoding, inputs])

    concat_input = layers.Concatenate(axis=-1)([inputs, pos_encoding])

    # CNN Branch:
    cnn = layers.Conv1D(filters=128, kernel_size=5, activation='relu', padding='same')(concat_input)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(0.3)(cnn)

    cnn = layers.Conv1D(filters=160, kernel_size=5, activation='relu', padding='same')(cnn)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(0.3)(cnn)

    cnn = layers.Conv1D(filters=192, kernel_size=5, activation='relu', padding='same')(cnn)
    cnn = layers.BatchNormalization()(cnn)
    cnn = layers.Dropout(0.3)(cnn)

    # DCNN Branch (replacing the LSTM branch):
    dcnn = layers.Conv1D(filters=128, kernel_size=5, dilation_rate=1, activation='relu', padding='same')(concat_input)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(0.3)(dcnn)
    
    dcnn = layers.Conv1D(filters=160, kernel_size=5, dilation_rate=2, activation='relu', padding='same')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(0.3)(dcnn)
    
    dcnn = layers.Conv1D(filters=192, kernel_size=5, dilation_rate=4, activation='relu', padding='same')(dcnn)
    dcnn = layers.BatchNormalization()(dcnn)
    dcnn = layers.Dropout(0.3)(dcnn)

    # Concatenate the two branches:
    combined = layers.Concatenate()([cnn, dcnn])

    combined = layers.Conv1D(128, kernel_size=1, activation='relu')(combined)
    combined = layers.BatchNormalization()(combined)
    combined = layers.Dropout(0.3)(combined)
    
    combined = layers.Conv1D(128, kernel_size=1, activation='relu')(combined)
    combined = layers.BatchNormalization()(combined)

    # Final classification layer applied at every time step:
    outputs = layers.Conv1D(num_classes, kernel_size=1, activation='softmax')(combined)

    model = Model(inputs=inputs, outputs=outputs)
    return model

parallel_hybrid_model = create_parallel_hybrid_model(5, 5000, 5)
parallel_hybrid_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
parallel_hybrid_model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 5000, 5)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_4 (Lambda)   │ (None, 5000, 5)   │          0 │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 5000, 10)  │          0 │ input_layer_4[0]… │
│ (Concatenate)       │                   │            │ lambda_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_36 (Conv1D)  │ (None, 5000, 128) │      6,528 │ concatenate_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_39 (Conv1D)  │ (None, 5000, 128) │      6,528 │ concatenate_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 128) │        512 │ conv1d_36[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 128) │        512 │ conv1d_39[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_28          │ (None, 5000, 128) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_31          │ (None, 5000, 128) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_37 (Conv1D)  │ (None, 5000, 160) │    102,560 │ dropout_28[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_40 (Conv1D)  │ (None, 5000, 160) │    102,560 │ dropout_31[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 160) │        640 │ conv1d_37[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 160) │        640 │ conv1d_40[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_29          │ (None, 5000, 160) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_32          │ (None, 5000, 160) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_38 (Conv1D)  │ (None, 5000, 192) │    153,792 │ dropout_29[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_41 (Conv1D)  │ (None, 5000, 192) │    153,792 │ dropout_32[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 192) │        768 │ conv1d_38[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 192) │        768 │ conv1d_41[0][0]   │
│ (BatchNormalizatio… │                   │            │                 

 Total params: 597,061 (2.28 MB)

 Trainable params: 594,629 (2.27 MB)

 Non-trainable params: 2,432 (9.50 KB)

In [10]:
### Callbacks Box ###

class TimeLimit(callbacks.Callback):
    def __init__(self, max_time_seconds):
        super().__init__()
        self.max_time_seconds = max_time_seconds
        self.start_time = None

    def on_train_begin(self, logs=None):
        self.start_time = time.time()

    # def on_batch_end(self, batch, logs=None):
    #     if time.time() - self.start_time > self.max_time_seconds:
    #         self.model.stop_training = True
    
    # def on_train_batch_end(self, batch, logs=None):  # ✅ Runs more frequently than `on_batch_end`
    #     elapsed_time = time.time() - self.start_time
    #     if elapsed_time > self.max_time_seconds:
    #         print(f"\n⏳ Time limit of {self.max_time_seconds} sec reached. Stopping training!")
    #         self.model.stop_training = True  # 🔥 Stops training mid-batch
    
    def on_train_batch_begin(self, batch, logs=None):
        elapsed_time = time.time() - self.start_time
        if elapsed_time > self.max_time_seconds:
            print(f"\n⏳ Time limit of {self.max_time_seconds} sec reached. Stopping training!")
            self.model.stop_training = True

    def on_epoch_end(self, epoch, logs=None):  # New method added
        if time.time() - self.start_time > self.max_time_seconds:
            self.model.stop_training = True
            
class DebugCallback(callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"\n🚀 Starting Epoch {epoch+1}")
        sys.stdout.flush()

    def on_batch_begin(self, batch, logs=None):
        if batch % 1000 == 0:
            print(f"🔄 Processing Batch {batch}")
            sys.stdout.flush()

    def on_batch_end(self, batch, logs=None):
        if batch % 1000 == 0:
            print(f"✅ Finished Batch {batch}")
            sys.stdout.flush()

    def on_epoch_end(self, epoch, logs=None):
        print(f"\n🏁 Epoch {epoch+1} Completed!")
        sys.stdout.flush()
        

class BestModelCheckpoint(callbacks.Callback):
    """
    Custom callback to save the best model dynamically.
    The model is named based on the Keras model's name.
    Tracks the best validation loss and updates the model file only when improved.
    """
    
    def __init__(self, save_dir="Test_Models", log_dir="best_model_logs"):
        super().__init__()
        self.save_dir = save_dir
        self.log_dir = log_dir
        self.best_val_loss = {}

        # Ensure directories exist
        os.makedirs(save_dir, exist_ok=True)
        os.makedirs(log_dir, exist_ok=True)

    def on_train_begin(self, logs=None):
        """Initialize tracking for this specific model when training starts."""
        model_name = self.model.name if self.model.name else "unnamed_model"
        self.model_path = os.path.join(self.save_dir, f"{model_name}_best.keras")
        self.log_path = os.path.join(self.log_dir, f"{model_name}_best_log.txt")

        # Load previous best validation loss if exists
        if os.path.exists(self.log_path):
            with open(self.log_path, "r") as f:
                try:
                    self.best_val_loss[model_name] = float(f.read().strip())
                except ValueError:
                    self.best_val_loss[model_name] = float("inf")
        else:
            self.best_val_loss[model_name] = float("inf")

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_loss = logs.get("val_loss")

        if val_loss is not None:
            model_name = self.model.name if self.model.name else "unnamed_model"

            if val_loss < self.best_val_loss[model_name]:
                self.best_val_loss[model_name] = val_loss  # Update best loss
                self.model.save(self.model_path)  # Save the model
                print(f"\n✅ New Best Model Saved ({model_name}) at Epoch {epoch+1} with val_loss: {val_loss:.5f}")

                # Update log file with new best loss
                with open(self.log_path, "w") as f:
                    f.write(str(self.best_val_loss[model_name]))
            else:
                print(f"\n🔹 Epoch {epoch+1}: No Improvement for {model_name} (Best val_loss: {self.best_val_loss[model_name]:.5f})")

    def reset(self):
        """Resets the best validation loss tracking (e.g., before a new loop run)."""
        self.best_val_loss = {}
        for file in os.listdir(self.log_dir):
            os.remove(os.path.join(self.log_dir, file))  # Clear log files


In [11]:
histories = {}
# tf.debugging.set_log_device_placement(True)
# The functions for creating models are written in jupyter boxes a few above
create_models = [create_cnn_model, create_dcnn_model, create_cnn_to_dcnn_model, create_dcnn_to_cnn_model, create_parallel_hybrid_model]
for model_name, model_creator in zip(["cnn", "dcnn", "cnn_to_dcnn", "dcnn_to_cnn", "parallel_hybrid"], create_models):
    print(f"\n🚀 Training {model_name}...\n")
    tf.debugging.set_log_device_placement(True)
    
    max_time_seconds = 8640  # 1 hour is 3600 seconds
    batch_size = 28
    epochs = 1000  # Set high enough to allow stopping by time
    steps_per_epoch = 6341

    time_limit_callback = TimeLimit(max_time_seconds=max_time_seconds)
    print('Compiling train dataset')
    train_dataset = prep_dataset_from_tfrecord("TestValTrain/train.tfrecord.gz",
                                    batch_size=batch_size, 
                                    compression_type='GZIP', 
                                    shuffled=True,
                                    shuffle_buffer=25000,
                                    total_records=177567,
                                    num_to_drop=19
                                    )
    print('Compiling val dataset')
    val_dataset = prep_dataset_from_tfrecord("TestValTrain/val.tfrecord.gz",
                                    batch_size=batch_size, 
                                    compression_type='GZIP', 
                                    shuffled=False,
                                    shuffle_buffer=25000,
                                    total_records=20890,
                                    num_to_drop=2,
                                    seed=42
                                    )

    # test_dataset = prep_dataset_from_tfrecord("TestValTrain/test.tfrecord.gz",
    #                                 batch_size=batch_size, 
    #                                 compression_type='GZIP', 
    #                                 shuffled=False,
    #                                 shuffle_buffer=25000,
    #                                 total_records=10446,
    #                                 num_to_drop=2,
    #                                 seed=42
    #                                 )
    print("Creating next model")
    model = model_creator(5, 5000, 5)
    model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(ignore_class= 0), metrics=[CustomNonZeroF1Score(num_classes=5, average='weighted')])
    model.summary()
    
    print('Fitting model')
    history = model.fit(
        train_dataset, 
        validation_data=val_dataset,
        # batch_size=batch_size,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        callbacks=[time_limit_callback, DebugCallback(), BestModelCheckpoint()]
    )

    print('Saving model...')
    model.save(f"Test_Models/{model_name}.keras")
    print(f"📁 Model {model_name} saved!")
    
    # 🔥 Clear GPU memory
    K.clear_session()
    gc.collect()
    print("Cleaned memory")


🚀 Training cnn...

Compiling train dataset
Compiling val dataset
Creating next model


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 5000, 5)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_5 (Lambda)   │ (None, 5000, 5)   │          0 │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_6       │ (None, 5000, 10)  │          0 │ input_layer_5[0]… │
│ (Concatenate)       │                   │            │ lambda_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_45 (Conv1D)  │ (None, 5000, 128) │      6,528 │ concatenate_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 128) │        512 │ conv1d_45[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_35          │ (None, 5000, 128) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_46 (Conv1D)  │ (None, 5000, 128) │     82,048 │ dropout_35[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 128) │        512 │ conv1d_46[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_36          │ (None, 5000, 128) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_47 (Conv1D)  │ (None, 5000, 160) │    102,560 │ dropout_36[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 160) │        640 │ conv1d_47[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_37          │ (None, 5000, 160) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_48 (Conv1D)  │ (None, 5000, 160) │    128,160 │ dropout_37[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 160) │        640 │ conv1d_48[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_38          │ (None, 5000, 160) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_49 (Conv1D)  │ (None, 5000, 192) │    153,792 │ dropout_38[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5000, 192) │        768 │ conv1d_49[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_39          │ (None, 5000, 192) │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_50 (Conv1D)  │ (None, 5000, 192) │    184,512 │ dropout_39[0][0]

 Total params: 704,325 (2.69 MB)

 Trainable params: 701,893 (2.68 MB)

...


🏁 Epoch 24 Completed!

🔹 Epoch 24: No Improvement for functional (Best val_loss: 0.19587)
6341/6341 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 0.0000e+00 - non_zero_f1: 0.0000e+00 - val_loss: 0.2231 - val_non_zero_f1: 0.0019

🚀 Starting Epoch 25
Epoch 25/1000


2025-02-04 13:30:27.742858: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 4830053258104170669
2025-02-04 13:30:27.742952: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 4210857637735185062


🔄 Processing Batch 0
✅ Finished Batch 0
1000/6341 ━━━━━━━━━━━━━━━━━━━━ 9:05 102ms/step - loss: 0.2151 - non_zero_f1: 0.0019🔄 Processing Batch 1000
✅ Finished Batch 1000
1844/6341 ━━━━━━━━━━━━━━━━━━━━ 7:38 102ms/step - loss: 0.2169 - non_zero_f1: 0.0019
⏳ Time limit of 8640 sec reached. Stopping training!
1845/6341 ━━━━━━━━━━━━━━━━━━━━ 7:38 102ms/step - loss: 0.2169 - non_zero_f1: 0.0019
🏁 Epoch 25 Completed!

🔹 Epoch 25: No Improvement for functional (Best val_loss: 0.19587)
6341/6341 ━━━━━━━━━━━━━━━━━━━━ 210s 32ms/step - loss: 0.2185 - non_zero_f1: 0.0019 - val_loss: 0.2246 - val_non_zero_f1: 0.0019


2025-02-04 13:33:57.556490: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 4830053258104170669
2025-02-04 13:33:57.556577: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 4210857637735185062


Saving model...
📁 Model parallel_hybrid saved!
Cleaned memory


Best val loss for each model was:

.23078 .19587 .2051 .2067 .2228

Pure dilation had best performance